# The script cleans up and normalizes the addresses and contact information

## 1. Load and clean up data

In [55]:
import pandas as pd
import numpy as np
import re
import funcs as normex
import random
import imp

imp.reload(normex)

pd.set_option('display.precision', 15)


def some(x, n):
    return x.ix[random.sample(x.index.tolist(), n)]

    
def split_address(df):
    df['street'] = df.rus_addr.str.lower()
    df['street'], df['addr'] = zip(*df.street.map(normex.address_splitter))

    
def preprocess(file, sheetname=0, is_test_mode=True, header=None, swap_addr_and_tel=False):
    df = pd.read_excel(file, sheetname=sheetname, header=header, encoding='utf-8').reset_index(drop=True)
#     df = pd.read_csv(file, header=header).reset_index(drop=True)
    
    df.Адрес.to_csv('streets_mixed.csv', index=True, encoding='utf-8')
    
    df.dropna(axis=0, how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    # df.drop(df.columns[4:], axis=1, inplace=True)
    # df.dropna(axis=1, how='all', inplace=True)

    
    #swap address and tel if misplaced
    if swap_addr_and_tel:
        swap_idx = df.Адрес.str.contains(r'^[\d\s-]+$', na=True) #index for rows to swap address and tel 
        df.loc[swap_idx, ['Адрес', 'Телефон 1']] = df[swap_idx][['Телефон 1','Адрес']].values
        df.Адрес.fillna(value='<no address>', inplace=True)
        df[df.Адрес.str.contains('<no address>')]
   
    return df


# df.Адрес[~df.Адрес.isin(ukr_addr.Адрес)]
# len(ukr_addr), len(df), len(rus_addr)
    
# raw_data = preprocess('sample.xlsx', 'DB', rus_addr_list='streets_rus_for_data_preprocessed.txt', is_test_mode=True, header=2)

raw_data = preprocess('База Укропчик.xlsx', is_test_mode=False, header=0)

[5232323, 43454454]


In [ ]:
import pandas as pd


def post_process_translated_streets(csv_file):
    addr = pd.read_csv(csv_file, header=None, names=['Адрес'], index_col=0)

    addr.Адрес.replace('\n', ' ', inplace=True)
    addr.Адрес.replace(to_replace='(П|п)роспект', value='просп.', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='(П|п)(Р|р)\.', value='просп. ', regex=True, inplace=True)    
    addr.Адрес.replace(to_replace='(П|п)-т', value='просп. ', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='^(В|в)ул(\.|\s)', value='ул. ', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='^(В|в)(\.|\s)', value='ул. ', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='^(Н|н)(\.|\s)?(аб)?.*Перемог.', value='ул. Набережная Победы', regex=True, inplace=True)

    addr.Адрес.replace(to_replace='\d*\D*м.Днепропетровск,', value='', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='Наб\.', value='Набережная ', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='НаБерезоваяя', value='Набережная ', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='Б-Р ', value='бульв. ', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='З.КосмодемъЯнськое', value='Зои Космодемьянской', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='[Пп]роспект', value='просп.', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='[Зз]атишна', value='Уютная', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='[Пп]охил', value='Наклонн', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='рокив', value='лет', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='Днипрельстанивская', value='Днепрельстановская', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='Водиев', value='Водителей', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='Зовнишня', value='Внешняя', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='Криворизька', value='Криворожская', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='Металургийна', value='Металургическая', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='Повстання', value='Востания', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='Тиниста', value='Тенистая', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='[Пп]івнічна', value='Северная', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='[Пп]ивнична', value='Северная', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='[Рр]адянская', value='Советская', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='[Рр]адянська', value='Советская', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='[Дд]ивізи[ия]', value='Дивизии', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='Стрілецька', value='Стрелецкая', regex=True, inplace=True)
    addr.Адрес.replace(to_replace='[Жж]овтня', value='Октября', regex=True, inplace=True)

    addr.to_csv('in/postprocessed_address.txt')

In [ ]:
def append_rus_address_column(df, rus_addr_list):
    rus_addr = pd.read_csv(rus_addr_list, header=None, names=['Адрес'], index_col=0, encoding='Windows-1251')
    df['rus_addr'] = rus_addr.Адрес
    

post_process_translated_streets('in/ukropchik_translated.txt')
append_rus_address_column(raw_data, rus_addr_list='in/postprocessed_address.txt')
split_address(raw_data)

## 3. Load address book, generate n-grams

In [2]:
import time

start = time.clock()

addr_book = pd.read_excel('address_book.xlsx', header=1, encoding='utf-8')
addr_book['street_lower'] = addr_book.Улица.str.lower()


print('time: ' + str(time.clock() - start))

time: 30.08024381983273


In [ ]:
import ngram
from collections import defaultdict

def default():
    return '<not found>'

def street_ngrams(addr_row):        
    addr_list = addr_row.Дом.map(str) +  '$' + addr_row.Квартира.map(str)    
    build_and_apt = [str(x) for x in addr_list.str.lower().tolist()]
    return ngram.NGram(build_and_apt, N=3)


# returns maping from lower case street names to numbers ngrams
def street_to_numbers_ngrams(addr_book):
    str_to_addr_ngram = defaultdict(default)
    
    unique_streets = addr_book.street_lower.unique()
    for st in unique_streets:        
        addr_row = addr_book[addr_book.street_lower == st]
        st_ngr = street_ngrams(addr_row)
        str_to_addr_ngram[st] = st_ngr    
    return str_to_addr_ngram


## street to number ngrams
start = time.clock()
str_ngram_dict = street_to_numbers_ngrams(addr_book)
print('street number ngrams time: ' + str(time.clock() - start))

In [ ]:
## street name ngrams
start = time.clock()
street_choices = [str(x) for x in addr_book.street_lower.unique()]
streets_G2 = ngram.NGram(street_choices, N=2)
print('street name ngrams time: ' + str(time.clock() - start))

In [ ]:
def correct_street(x, ngr):
    if 'error' in str(x):
        return ('<error>', 0)
    return ngr.search(x)[0]


def match_streets(x, streets_G2):
    start = time.clock()
    x['street'], x['str_score'] = zip(*x.street.apply(correct_street, args=(streets_G2,)))
    print('match streets time: ' + str(time.clock() - start))
    

def correct_number(addr_row, ngr_dict):
    # 6 - 'street' 
    # 7 - 'addr' 
    street, addr = addr_row[6], addr_row[7]
    ngr = ngr_dict[street]
    
    if pd.isnull(addr) or 'error' in addr:
        return ('<error>', '<error>', 0)
    
    result = ngr.search(addr)
    
    if not result:
        return ('<error>', '<error>', 0)
    
    number, score = result[0]
    build, apt = number.split('$') 
    return (build, apt, score)


def match_numbers(x, str_ngram_dict):    
    start = time.clock()
    x['build'], x['apt'], x['num_score'] = zip(*df.apply(correct_number, args=(str_ngram_dict,), axis=1, raw=True))
    print('match numbers time: ' + str(time.clock() - start))
    
    
def correct_tel(x):    
    start = time.clock()
    x['тел. 1'], x['тел. 2'] = x['тел. 1'].astype(object), x['тел. 2'].astype(object)
    phones = x.apply(lambda x: normex.parse_tel(x['тел. 1'], x['тел. 2']), axis=1)
    x['tel_1'], x['tel_2'], x['tel_3'] = zip(*phones)
    print('correct tel time: ' + str(time.clock() - start))
    

groups = raw_data.groupby(by='Источник', sort=False, group_keys=True)
lower_to_upper_str = addr_book.groupby(by=['street_lower', 'Улица'])

group_files = list()
for x in groups.groups:
    g = groups.get_group(x)
    f_name = 'data/%s.csv' % g.Источник.unique()[0]
    g.to_csv(f_name)
    group_files.append(f_name)
    
print('Saved groups in separate files')
print('============================================')


out_files = list()    
for f in group_files:
    if 'Укропчик' in f:
        continue
    df = pd.read_csv(f, encoding='Windows-1251', header=0, index_col=0)
    print('Loaded file ' + f)
    match_streets(df, streets_G2)
    match_numbers(df, str_ngram_dict)   
    df.street.replace(dict(lower_to_upper_str.groups.keys()), inplace=True)
    correct_tel(df)
    print('Finished processing: ' + f)
    f_name = 'out/' + f
    df.to_csv(f_name)
    out_files.append(f_name)
    print('Saved to: ' + f_name)
    print('============================================')

ukropchik = pd.read_csv('data/Укропчик.csv', encoding='Windows-1251', header=0, index_col=0)    
correct_tel(ukropchik)
ukropchik_out_file = 'out/data/Укропчик.csv'
ukropchik.to_csv(ukropchik_out_file)
out_files.append(ukropchik_out_file)

out_dfs = list()
for f in out_files:
    partial = pd.read_csv(f, encoding='Windows-1251', header=0, index_col=0)
    out_dfs.append(partial)
    
out = pd.concat(out_dfs)

index = ['Источник', 'ФИО', 'Адрес', 'тел. 1', 'тел. 2',
         'rus_addr', 'addr', 'street', 'build', 'apt', 'tel_1', 'tel_2', 'tel_3',
         'num_score', 'str_score']
out = out.reindex_axis(index, axis=1)
out.to_csv('out/data/final.csv')    


# df_arr = np.array_split(df, 7)
# for i, x in enumerate(df_arr):
#     fname = 'data/mixed_streets_production%i.txt' % i
#     x.to_csv(fname, encoding='utf-8')

## Collect outputs to single file

In [ ]:
import os
import pandas as pd

out_dir = 'out/data/'
out_dfs = list()
for f in os.listdir(out_dir):
    partial = pd.read_csv(out_dir + f, encoding='Windows-1251', header=0, index_col=0)
    out_dfs.append(partial)
    
out = pd.concat(out_dfs)

index = ['Источник', 'ФИО', 'Адрес', 'тел. 1', 'тел. 2',
         'rus_addr', 'addr', 'street', 'build', 'apt', 'tel_1', 'tel_2', 'tel_3',
         'num_score', 'str_score']
out = out.reindex_axis(index, axis=1)
# out.to_csv('out/data/final_test.csv')

## Translate street names (don't abuse, might be Google-banned )

In [ ]:
import goslate
from urllib import request


# proxy_handler = request.ProxyHandler({"http" : "http://176.31.119.64:8080"})
# proxy_opener = request.build_opener(request.HTTPHandler(proxy_handler), request.HTTPSHandler(proxy_handler))
# gs = goslate.Goslate(opener=proxy_opener)

with open('mixed_streets_production.txt', 'r', encoding='utf-8') as f:
     novel_text = f.read()
# gs = goslate.Goslate()
# streets = [str(x) for x in raw_data.Адрес.tolist()]
gs = goslate.Goslate()
translation = gs.translate(novel_text, 'ru', source_language='uk')
translation = list(streets_rus)

In [ ]:
from pyspark import  SparkContext, SQLContext
sc = SparkContext( 'local', 'pyspark')

## Lookup and match street names

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time
import ngram

street_choices = [str(x) for x in addr_lookup.Улица.tolist()]
G2 = ngram.NGram(street_choices, N=2)

def match_scores(st):
    return [(fuzz.token_set_ratio(st, normalized_st), normalized_st, st) for normalized_st in street_choices]

def reducer(score1, score2):  
    print(score1, score2)    
    return max(score1, score2, key=lambda x: x[0])
#     scores = [(fuzz.token_set_ratio(raw, normal), normal) for normal in street_choices]
#     return max(scores, key=lambda x: x[0])[1]


start = time.clock()

# dirty_streets = sc.parallelize(raw_data.street.head(10).str.encode(encoding='utf-8').tolist())
# r = dirty_streets.flatMap(match_scores).reduceByKey(reducer).collect()

print('time: ' + str(time.clock() - start))


# streets_dictionary = prepare_match_list(street_choices)




start = time.clock()
matched_streets = list()
# for raw in raw_data.street.tolist():    
#     scores = [(fuzz.token_set_ratio(raw, normal), normal) for normal in G.search(raw, threshold=0.3)]
#     if (scores):
#         matched_streets.append(max(scores, key=lambda x: x[0])[1])
#     match = process.extractOne(raw, street_choices)
#     matched_streets.append(match[0])
    
# print(matched_streets)
print('time ngram on test data: ' + str(time.clock() - start))


# pd.isnull(addr_lookup['Unnamed: 1'])
# addr_lookup['Unnamed: 1'].iloc[1]

# df = raw_data.dropna(how='all')
# # df = raw_data.drop(raw_data.index[1])
# df = df[df.Адрес.str.contains(u'Правд')==True]
# df



choices = ['просп. Воронцова', 'просп. Гагарина', 'просп. Газеты "Правда"', 'просп. Героев', 'просп. Ильича', 
           'просп. Карла Маркса', 'просп. Кирова', 'просп. Металлургов', 'просп. Мира', 'просп. Олимпийский',
           'просп. Петровского', 'просп. Пушкина', 'просп. Свободы', 'просп. Сергея Нигояна', 'просп. Труда',
           'ул. Правды', 'ул. Юдина', 'ул. Набережная',  'ул. Набережная В.И.Ленина', 'пер. Пролетарской Победы', 
           'ул. Набережная Заводская', 'ул. Набережная им. Ленина', 'ул. Набережная Победы', 
           'ул. Мира', 'ул. Железнодорожная', 'ул. Дорожная', 'пер. Пролетарской Победы']    

# process.extract(u'ул. Железнодорожна', choices)
# fuzz.partial_token_sort_ratio(u'Наб. Победы', 'ул. Набережная Победы')
# fuzz.partial_ratio(u'Наб. Победы', 'ул. Набережная Победы')


# sorted([(fuzz.QRatio('ул.Малиновського,', normal), normal) for normal in street_choices], reverse=True)

start = time.clock()
[fuzz.token_set_ratio('ул.Малиновського,', normal) for normal in street_choices]
print('time: ' + str(time.clock() - start))

start = time.clock()
[G2.find(x) for x in raw_data.street.tolist()]
print('time bi-gram: ' + str(time.clock() - start))

## String matching heuristics

In [37]:
import streets
imp.reload(streets)

parser = streets.AddressParser(addr_book_file='address_book.xlsx', street_field='Улица', build_field='Дом', apt_field='Квартира')


In [35]:
import funcs as numex
imp.reload(numex)

def correct_tel(x):    
    start = time.clock()
    x['тел. 1'], x['тел. 2'] = x['тел. 1'].astype(object), x['тел. 2'].astype(object)
    phones = x.apply(lambda x: numex.parse_tel(x['тел. 1'], x['тел. 2']), axis=1)
    x['tel_1'], x['tel_2'], x['tel_3'] = zip(*phones)
    print('correct tel time: ' + str(time.clock() - start))

[5232323, 43454454]
27кв 92 ->> ('27', '92')
73 кор 6 кв 55 ->> ('73/6 ', '55')
4гурт, кв.141,517,518 ->> ('4', '141,517,518')
4гурт, кв.517,518,141 ->> ('4', '517,518,141')
7, кв.1,2,3,4 ->> ('7', '1,2,3,4')
7, кв.1/2/3/4 ->> ('7', '1,2,3,4')
19а, кв.115 ->> ('19-а', '115')
2, кв. 10-а ->> ('2', '10,а')
8, кв.267 ->> ('8', '267')
, 22, корп.7, кв.61 ->> ('22/7', '61')
18, кв. 1а) ->> ('18', '1а')
60, кв. 259) ->> ('60', '259')
83, корп.3, кв.56 ->> ('83/3', '56')
38 кв 180 ->> ('38', '180')
122/148 ->> ('122', '148')
164а, кв. 37 ->> ('164-а', '37')
97а/51 ->> ('97-а', '51')
, буд. 57, кв. 154 ->> ('57', '154')
20б, кв.77 ->> ('20-б', '77')
, 3, кв.159 ->> ('3', '159')
, 51, корп.3, кв.86 ->> ('51/3', '86')
70б, кв.52 ->> ('70-б', '52')
73, кв.110 ->> ('73', '110')
77, кв.48 ->> ('77', '48')
46, кв. 441 ->> ('46', '441')
127а/68 ->> ('127-а', '68')
time: 0.00136244209807046
sample size: 26


In [39]:
groups = raw_data.groupby(by='Источник', sort=False, group_keys=True)
lower_to_upper_str = addr_book.groupby(by=['street_lower', 'Улица'])
    
    

group_files = list()
for x in groups.groups:
    g = groups.get_group(x)
    f_name = 'data/%s.csv' % g.Источник.unique()[0]
    g.to_csv(f_name)
    group_files.append(f_name)
    
print('Saved groups in separate files')
print('============================================')


skip_files = ['Укропчик', 
#               'oldmen Ленинский', 'ЛФК', 'oldmen Красногвардейский', 'ЕБА', 'oldmen АНД', 
#               'oldmen Индустриальный', 'ИЗБ', 'общ. Прием', 'Бабушки', 'Инвалиды', 'oldmen Кировский'
             ]
out_files = list()    
for f in group_files:
    if any(skip in f for skip in skip_files):
        continue
    df = pd.read_csv(f, encoding='Windows-1251', header=0, index_col=0)
    print('Loaded file ' + f)
    
    start = time.clock()
    df['street'], df['str_score'] = zip(*df.street.apply(lambda s: parser.parse_street_name(s)))
    print('match streets time: ' + str(time.clock() - start))
                                      
    start = time.clock()
#     df['build'], df['apt'], df['num_score'] = zip(*df.apply(lambda r: parser.parse_build_number(r.street, r.addr), axis=1, raw=True))
    df['build'], df['apt'] = zip(*df.apply(lambda r: numex.parse_build_number(r.addr), axis=1, raw=True))
    print('match numbers time: ' + str(time.clock() - start))
    
    df.street.replace(dict(lower_to_upper_str.groups.keys()), inplace=True)
    correct_tel(df)
    print('Finished processing: ' + f)
    f_name = 'out/' + f
    df.to_csv(f_name)
    out_files.append(f_name)
    print('Saved to: ' + f_name)
    print('============================================')

ukropchik = pd.read_csv('data/Укропчик.csv', encoding='Windows-1251', header=0, index_col=0)    
correct_tel(ukropchik)
ukropchik_out_file = 'out/data/Укропчик.csv'
ukropchik.to_csv(ukropchik_out_file)
out_files.append(ukropchik_out_file)

out_dfs = list()
for f in out_files:
    partial = pd.read_csv(f, encoding='Windows-1251', header=0, index_col=0)
    out_dfs.append(partial)
    
out = pd.concat(out_dfs)

index = ['Источник', 'ФИО', 'Адрес', 'тел. 1', 'тел. 2',
         'rus_addr', 'addr', 'street', 'build', 'apt', 'tel_1', 'tel_2', 'tel_3',
         'num_score', 'str_score']
out = out.reindex_axis(index, axis=1)
out.to_csv('out/data/final.csv')    



Saved groups in separate files
Loaded file data/oldmen Ленинский.csv
match streets time: 8.922666229911556
match numbers time: 3.3525932584307157
correct tel time: 7.326861662091687
Finished processing: data/oldmen Ленинский.csv
Saved to: out/data/oldmen Ленинский.csv
Loaded file data/ЛФК.csv
match streets time: 0.53546711322997
match numbers time: 0.007501128951844294
correct tel time: 0.020706468560092617
Finished processing: data/ЛФК.csv
Saved to: out/data/ЛФК.csv
Loaded file data/oldmen Красногвардейский.csv
match streets time: 3.1160876786598237
match numbers time: 2.055904599976202
correct tel time: 4.775135046831565
Finished processing: data/oldmen Красногвардейский.csv
Saved to: out/data/oldmen Красногвардейский.csv
Loaded file data/ЕБА.csv
match streets time: 16.129778382928635
match numbers time: 0.19285654417035403
correct tel time: 0.6922381853291881
Finished processing: data/ЕБА.csv
Saved to: out/data/ЕБА.csv
Loaded file data/oldmen АНД.csv
match streets time: 6.0684111855

In [29]:
# u = raw_data.groupby(by='addr')
# k = u.groups.keys()

# a = pd.DataFrame({'addr' : list(k)})

# a.to_csv('unique_addr.csv')

# a.addr.to_csv('unique_addr.csv')

# np.savetxt('unique_addr.txt', u, delimiter='$$', fmt='%s')

raw_data.iloc[3189]

Источник                                 ЕБА
ФИО             Герасимов Богдан Анатольевич
Адрес       ул. Червоного Казачеста, 27кв 92
тел. 1                            0989335335
тел. 2                                   NaN
rus_addr    ул. Червоного Казачеста, 27кв 92
street             ул. червоного казачеста, 
addr                                 27кв 92
Name: 3189, dtype: object

In [53]:
# raw_data[raw_data.addr.str.contains('кв.*\d-.+')].addr

regex = re.compile(r'\d+')
raw_data[raw_data.addr.str.contains('\d+-[\d\w]+.*кв.*')][['addr', 'street']]
raw_data[raw_data.addr.map(lambda x: len(regex.findall(x)) > 3)][['addr', 'street']]


,addr,street
3097,"63 кв 19, прописан ул. с.ковалевской, 67 кв 113","ул. мандрыковская,"


In [54]:
raw_data

,Источник,Адрес,ФИО,тел. 1,rus_addr,street,addr
0,Укропчик,"вул. Дємєнтьєва, 2/71",Абакумова Ірина Олексіївна,066-574-01-31,"ул.Караваева,15 кв.21","ул.караваева,",15 кв.21
1,Укропчик,"ул.Гагарина, 37 кв.4 (Першотравенск)",Абальмасова Лариса Миколаївна,502240878,"ул.Коксовая, 12а кв.218","ул.коксовая,",12а кв.218
2,Укропчик,"пр.Гагарина, д37 кв.4 Першотравенск",Абальмасова Лариса Миколаївна,502240878,"просп. Кирова,109 кв.17","просп. кирова,",109 кв.17
3,Укропчик,Наб. Перемоги 130/1/189,Абамусова Наталія Вікторівна,675688508,"пер.Л.Мокиевской,2а/146","пер.л.мокиевской,",2а/146
4,Укропчик,в.Красночечелівська//б.46 кв.22,Аббасова Ганна Володимирівна//,500514364,просп. Мира73/39,просп. мира,73/39
5,Укропчик,"п-т Ілліча, буд. 21, кв. 75",Абдулахатов Абдукаюм Абдусаматович Абддулахато...,0960414961 0960414963,"ул. Щербины, 21/148","ул. щербины,",21/148
6,Укропчик,ул.Володарского 109а,Абражан Лидия Сергеевна//Абражан Игорь Анатоль...,501926152,"ул.Данилы Нечая,13/124","ул.данилы нечая,",13/124
7,Укропчик,"Запорізьке шосе, б.40, кв.357",Абрамова Ліана Анатоліївна Абрамов Володимир Ю...,994695916; 0666850000,"ул.Данилы Нечая,19/20","ул.данилы нечая,",19/20
8,Укропчик,"Пр. Миру, 71/62",Дмитро Володимирович,097-361-00-41,"ул.Рогалёва,11 кв.1","ул.рогалёва,",11 кв.1
9,Укропчик,Вул.Макарова 29/7,Абрамова Ганна Владимирівна – //Абрамов Сергій...,096-212-00-93,"ж/м Тополь 2, дом.22 кв.45",ж/м тополь 2,", дом.22 кв.45"


In [ ]:
df = raw_data
    print('Loaded file ' + f)
    
    start = time.clock()
    df['street'], df['str_score'] = zip(*df.street.apply(lambda s: parser.parse_street_name(s)))
    print('match streets time: ' + str(time.clock() - start))
                                      
    start = time.clock()
#     df['build'], df['apt'], df['num_score'] = zip(*df.apply(lambda r: parser.parse_build_number(r.street, r.addr), axis=1, raw=True))
    df['build'], df['apt'] = zip(*df.apply(lambda r: numex.parse_build_number(r.addr), axis=1, raw=True))
    print('match numbers time: ' + str(time.clock() - start))
    
    df.street.replace(dict(lower_to_upper_str.groups.keys()), inplace=True)
    correct_tel(df)
    print('Finished processing: ' + f)
    f_name = 'out/' + f
    df.to_csv(f_name)